## 필요한 패키지 임포트 및 메소드 생성

In [ ]:
## 패키지 설치
# pip install google-api-python-client

In [3]:
import pandas as pd
# anaconda환경 : conda install -c conda-forge google-api-python-client
from googleapiclient.discovery import build

def read_api_key():
    infile = open('C:/Users/4ser/Desktop/이도원/기타잡것/key_data/youtube_api_key_1.txt', 'r')
    api_key = infile.readline()
    return api_key

In [4]:
video_id = 'tZolEtoQyuY'
comments = list()
api_obj = build('youtube', 'v3', developerKey=read_api_key())

## response

- response 1회당 1건씩 API 할당량 소모. while문은 maxResults 수치에 따라 다름. 예시) maxResults=100 : 할당량 약 4회
- part='snippet,replies'에서 snippet은 json파일의 최상단 key값이라 생각하면 됨. (필수조건)
- maxResults=10000은 최대로 가져올 데이터의 갯수를 10000개로 제한.
- 이렇게 가져온 response는 'items'라는 key값을 가지며, `response['items']`는 JSON파일로 나타냄.

In [5]:
# https://www.youtube.com/watch?part='snippet,replies'&vieoid='asdasd'&maxResults=100이와 같이 요청을 함 snippet필수속성, replies댓글
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=1000).execute() 

In [6]:
while response:
    for item in response['items']:        
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

In [7]:
response['items']

[{'kind': 'youtube#commentThread',
  'etag': 'cac_RZOGXNWPSAlOe_0JP0Be5Qk',
  'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
  'snippet': {'videoId': 'tZolEtoQyuY',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': '4xcAztyLhqHNQfGOQgvvH6KXa88',
    'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
    'snippet': {'videoId': 'tZolEtoQyuY',
     'textDisplay': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'textOriginal': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'authorDisplayName': '트오밥',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJUC18-1DuTjvb8N-WmaM6LC-1iKONdQF4FwHw=s48-c-k-c0x00ffffff-no-rj',
     'authorChannelUrl': 'http://www.youtube.com/channel/UCrNbLJUDU5BE8tTU_JFL8gg',
     'authorChannelId': {'value': 'UCrNbLJUDU5BE8tTU_JFL8gg'},
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 9,
     'publishedAt': '2022-12-04T10:23:27Z',
     'updatedAt': '2022-12-04T10:23:27Z'}},
   'canReply': True,
   'totalReplyCount': 0,
   'isPublic': True}},
 {'ki

## 내보내기

### 데이터프레임화 및 csv로 내보내기

In [8]:
df = pd.DataFrame(comments)
df.columns = ['comment', 'author', 'date', 'num_likes']
# df.to_csv('results.xlsx', index=None)
df

,comment,author,date,num_likes
0,"<a href=""https://www.youtube.com/watch?v=tZolE...",박민수,2023-02-12T13:12:25Z,0
1,"<a href=""https://www.youtube.com/watch?v=tZolE...",우름,2023-02-03T17:30:47Z,0
2,라꼰 시즌4 얼릉 해줘여 ㅠ,ㅎㅎㅎ,2023-02-03T02:19:01Z,0
3,먹방 방송중 자기들이 먹기 전에 스테프들 먼저 챙겨 주는 먹방은 처음보네요<br>침...,youngjin na,2023-02-02T21:27:53Z,1
4,아 진짜 만화가들의 저 상상력과 입담...쵝오야..끝까지 순수함을 지켜죠 ㅠ..,Punk-in Pie ASMR,2023-02-01T03:33:36Z,0
...,...,...,...,...
778,과연 몇이나 될까…,개삭수네,2022-12-14T09:05:53Z,0
779,장난치고 안먹으면 쳐맞아야 되지만<br>다먹음 ㅋㅋㅋㅋㅋㅋㅋ,Ne Beul,2022-12-07T08:33:24Z,1
780,하지만 맛있었죠?,두부,2022-12-05T14:57:31Z,2
781,탈모팸 1쉬,ㄷㅎㄱㅈㅇ,2022-12-05T05:47:38Z,0


In [11]:
df.sort_values(by='num_likes', ascending=False)

,comment,author,date,num_likes
776,&quot;음식 가지고 장난치는 아재팸 1위&quot;,GeeChan Lee,2022-12-04T10:01:00Z,3323
568,다 커서도 이런 친구를 만날수 있다는게 넘 부럽다,크로커스,2022-12-04T10:51:57Z,1073
711,아 오늘 일요일이여서 기분 별로 였는데 저녁 보면서 보는 침착맨 개꿀잼이다...,물먹는미역,2022-12-04T10:05:22Z,1051
380,침펄이 풍한테 형형거리면서 노는거 왜케 귀엽냐,Cool태수,2022-12-04T13:41:27Z,1022
625,"<a href=""https://www.youtube.com/watch?v=tZolE...",강한별,2022-12-04T10:31:06Z,950
...,...,...,...,...
280,생신축하드려요,규민,2022-12-04T16:59:23Z,0
279,"<a href=""https://www.youtube.com/watch?v=tZolE...",XXX MADunknown,2022-12-04T17:02:44Z,0
278,"<a href=""https://www.youtube.com/watch?v=tZolE...",ㅡ,2022-12-04T17:09:13Z,0
277,칙촉이랑 제크 사오래요 왜케 얄밉냐ㅋㅋ,와플,2022-12-04T17:12:08Z,0


## JSON 파일로 내보내기

In [12]:
import json

# response['items'] 변수
items = response['items']

# 저장할 파일 경로
file_path = './items.json'

# json 파일로 내보내기
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(items, json_file)